In [1]:
import numpy as np
import pandas as pd
from tzwhere import tzwhere
from datetime import datetime, timezone
import timezonefinder, pytz

Converting time to UTC using the timezone string from the latitude and longitude information.

In [2]:
sightings = pd.read_pickle('sightings_add_UTC')

sightings['Date_Time'] = pd.to_datetime(sightings['Posted'])
tf = timezonefinder.TimezoneFinder()
sightings['timezone_str']  = sightings.apply(lambda x: tf.certain_timezone_at(lat=x.Lat, lng=x.Lon), axis = 1)

In [5]:
Time_UTC = []
for t,dt in zip(sightings.timezone_str, sightings.Date_Time): 
    try:
        local = pytz.timezone(t)
        naive = datetime.strptime(str(dt), "%Y-%m-%d %H:%M:%S")
        local_dt = local.localize(naive, is_dst=None)
        utc_dt = local_dt.astimezone(pytz.utc)
        Time_UTC.append(utc_dt)
    except:
        Time_UTC.append(np.datetime64('NaT'))

sightings['Time_UTC'] = Time_UTC
sightings[sightings['Time_UTC'].isnull()]

,Date_Time,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary,Detail_Summary_nltk,Notes,Country,City,State,Location,Lat,Lon,timezone_str,Time_UTC


There was an issue with the two digit year values all converting to 

In [4]:
#correcting 1900s to 2000s issue with date column
def fix_date(x):
    if x.year > 2021:
        return datetime(x.year - 100, x.month, x.day, x.hour, x.minute, x.second, x.microsecond, tzinfo=timezone.utc)
    else:
        return x
    

sightings['Time_UTC']=sightings['Time_UTC'].apply(fix_date)
sightings['Time_UTC']=pd.to_datetime(sightings['Time_UTC'], utc=True)

sightings.head()

,Date_Time,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary,Detail_Summary_nltk,Notes,Country,City,State,Location,Lat,Lon,timezone_str,Time_UTC
0,2021-04-23,Circle,9 minutes,Very strange ((NUFORC Note: Rocket launch f...,4/23/21,http://www.nuforc.org/webreports/162/S162815.html,\nVery strangeI have recorded a video of this ...,strange recorded video fighting,NaN,United States,Blackshear,GA,"Blackshear, GA United States",31.30443,-82.240580,America/New_York,2021-04-23 04:00:00+00:00
1,2021-04-23,Circle,Seconds,Ball in the sky ((NUFORC Note: Rocket launc...,4/23/21,http://www.nuforc.org/webreports/162/S162814.html,\nBall in the skyObject appears as a white bal...,ball skyobject appears white ball vapor screwi...,NaN,United States,Thompson,VA,"Thompson, VA United States",37.78198,-79.974385,America/New_York,2021-04-23 04:00:00+00:00
2,2021-04-23,Light,5 minutes,I was driving and saw something strange in the...,4/23/21,http://www.nuforc.org/webreports/162/S162822.html,\nI was driving and saw something strange in t...,driving saw something strange sky pulled car i...,NaN,United States,Vero Beach,FL,"Vero Beach, FL United States",27.63885,-80.393960,America/New_York,2021-04-23 04:00:00+00:00
3,2021-04-23,Light,3 minutes,2 extremely bright lights appeared over east c...,4/23/21,http://www.nuforc.org/webreports/162/S162824.html,\n2 extremely bright lights appeared over east...,2 extremely bright light appeared east coast n...,NaN,United States,Saint Augustine,FL,"Saint Augustine, FL United States",29.89469,-81.314520,America/New_York,2021-04-23 04:00:00+00:00
4,2021-04-23,Cone,>5 minutes,A cone of light coming from the sky unlike any...,4/23/21,http://www.nuforc.org/webreports/162/S162819.html,\nA cone of light coming from the sky unlike a...,cone light coming sky unlike anything ever see...,NaN,United States,Durham,NC,"Durham, NC United States",35.99542,-78.896440,America/New_York,2021-04-23 04:00:00+00:00


In [6]:
fileName = 'sightings_cleaned_UTC.pkl'
sightings.to_pickle(fileName, protocol = 4)